## Building a classifier model for the Multivariate Time Series data using LSTM in Keras


This is a three-class classification problem for the given multivariate time series data. The "goal" column has three values 0 (no event), 1 (event of type 1), and 2 (event of type 2). Although we can treat events 1 and 2 as one "positive" class and 0 as the "negative" class, we will build a 3-class classifier and create a confusion matrix.  

We use LSTM, which is a variation of RNN, for building our network.

#### About the dataset:
There are 17 columns in total, of which the first two columns are the IDs for each timeseries (to mark the beginning and end of each timeseries), and are therefore discarded. The next 14 columns are the variables listed in time in equal intervals, and the last column is the "goal" column. There are 89 samples in each time series. For example, rows 2 to 90 represent one time series, rows 91 to 178 represent the next time series, and so on.


#### Lets start by importing the necessary modules and loading the raw dataset:

In [1]:
# univariate lstm example
from numpy import array
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
from pandas import read_excel, DataFrame, concat
# split a univariate sequence into samples
from iteration_utilities import deepflatten
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('Ethanol.csv', header = None)
values = df.values



Nc = values.shape[1] # number of columns

values = values.astype('float32') # ensuring all the data is float

x_Train = df.drop(df.columns[[0]], axis=1)
y_Train = df[df.columns[[0]]].values
mylist = []
for i in range(0,x_Train.shape[0]):
    mylist.append(np.ones(x_Train.shape[1])*y_Train[i])
mylist = pd.DataFrame(mylist)
mylist = pd.DataFrame(mylist.stack().to_frame().values)

df2 = pd.DataFrame(x_Train.stack().to_frame().values)
scaled = pd.concat([df2, mylist], axis=1)
values=scaled.values

Using TensorFlow backend.


In [2]:
scaled.shape

(1758004, 2)

Note that we normalized all the features so that they range between 0 and 1.

Now we need to frame the raw dataset into a supervised learning problem.
We set timestep equal to 1 (t_prev = 1), which means that we only look at the data in the previous row to train and predict the class.

In [3]:
##############################################################
# framing the problem as supervised learning

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame the time series as a supervised learning dataset.
    
    Arguments:
        data:    Sequence of observations as a list or NumPy array.
        n_in:    Number of lag observations as input (X).
        n_out:   Number of observations as output (y).
        dropnan: Boolean, whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """

    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # future sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg

t_prev = 1 # timestep
t_next = 1
reframed = series_to_supervised(scaled, t_prev, t_next)
print("reframed.shape before drop =", reframed.shape)
# dropping the columns that we don't want to predict
reframed.drop(reframed.columns[[1,2,3]], axis=1, inplace=True)
print("reframed.shape after  drop =", reframed.shape)


reframed.shape before drop = (1758003, 4)
reframed.shape after  drop = (1758003, 1)


In [4]:
reframed.head()

,var1(t-1)
1,-0.936
2,-0.936
3,-0.936
4,-0.936
5,-0.936


Now we will split the data into train/dev and test sets.

We use 60% of the data for training, of which 20% is used for evaluation purposes (to make sure we are not overfitting). The rest of the data (40%) will be set aside to test the accuracy of the model.

Note: we could also use 50% or 40% of the data for training/development purposes and use the remaining 50% or 60% of the data for test. The outcome would be very similar.

In [6]:
#############################################################################
# splitting the data into train and test sets
from sklearn.preprocessing import OneHotEncoder
one =  OneHotEncoder()


# taking 60% of the data for train/dev, and the remaining 40% for test
n_train = 1751 * int( reframed.values.shape[0]/1751 * 0.60) #each time series has 89 entities

# inputs
train_X = np.array(reframed.values[:n_train, :])
test_X  = np.array(reframed.values[n_train:, :])
# outpts (making sure outputs are arrays of integers)
last_column = values.shape[1] - 1 # last column (goal) in the raw dataset

train_Y = np.array(values[t_prev:n_train+t_prev, last_column], dtype=int)
test_Y  = np.array(values[t_prev+n_train:   , last_column], dtype=int)

# Lets convert Y into a "one-hot representation" to make it suitable for the softmax classifier.
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
train_Y = one.fit_transform(train_Y.reshape(-1, 1)).toarray()
test_Y  = one.fit_transform(test_Y.reshape(-1, 1)).toarray();
# now each row in Y will be a one-hot vector representing each class (0, 1, or 2)

# reshaping the input data to be 3D for Keras [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], t_prev, train_X.shape[1]))
test_X  = test_X.reshape( (test_X.shape[0],  t_prev,  test_X.shape[1]))
print("train_X.shape =", train_X.shape)
print("test_X.shape  =", test_X.shape)
print("train_Y.shape =", train_Y.shape)
print("test_Y.shape  =", test_Y.shape)


train_X.shape = (1054102, 1, 1)
test_X.shape  = (703901, 1, 1)
train_Y.shape = (1054102, 4)
test_Y.shape  = (703901, 4)


c:\users\usuario\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\usuario\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence thi

In [7]:
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

Now we will design the LSTM model in Keras. We use 32 hidden units for the LSTM layer. For optimization, we use Adam algorithm with a learning rate of 0.01, $\beta_1$=0.9, $\beta_2$=0.999, and a decay rate of 0.01. We set the number of epuchs to 4 (higher values result in overfitting).

#### The most important hyperparameters in our model are:
- number of hidden units in LSTM
- learning rate
- timestep (t_prev)
- number of epuchs

The values for the chosen hyperparameters above lead to satisfactory results.

In [23]:
input_shape

(1, 1)

In [8]:
#############################################################################
# building the model in Keras

from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

# designing the network
input_shape = (train_X.shape[1], train_X.shape[2])

model = Sequential()
model.add(LSTM(64, activation='relu',input_shape=input_shape))
#model.add(LSTM(128, activation='relu'))
model.add(Dense(4))
model.add(Activation('softmax'))
#lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01
opt = optimizers.Adam()
### optimizer='adam'
### loss='categorical_accuracy', 'msle'
### metrics=['accuracy'], ['msle', 'mae']
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# fitting the network
#Trained = model.fit(train_X, train_Y, epochs=5, validation_data=(test_X, test_Y), verbose=2, shuffle=False)
Trained = model.fit(train_X, train_Y, epochs=50, validation_split = 0.2, verbose=2, shuffle=False)

# plotting history
pyplot.xlabel('Epoch')
pyplot.ylabel('Loss')
pyplot.plot(Trained.history['loss'],     label='train')
pyplot.plot(Trained.history['val_loss'], label='eval')
pyplot.legend()
pyplot.show()

# summary of the model
model.summary()


Train on 843281 samples, validate on 210821 samples
Epoch 1/50
 - 36s - loss: 1.1319 - acc: 0.5404 - val_loss: 2.0786 - val_acc: 0.2276
Epoch 2/50


KeyboardInterrupt: 

The accuracy of the training and development sets, based on the categorical crossentropy loss, is found to be 99.71% and 99.63%, respectively.

We now evaluate the model to test its accuracy and to create a confusion matrix:

In [95]:
#############################################################################
# evaluating the model

from sklearn.metrics import confusion_matrix, classification_report

Test = model.evaluate( test_X, test_Y, verbose=1 )
print()
print("Test Loss: ", Test[0])
print("Accuracy:  ", Test[1])
print()

Predict = model.predict(test_X)
#print("Predict.shape =", Predict.shape)

Y_pred = np.argmax(Predict, axis=1)
Y_true = np.argmax(test_Y,  axis=1)

#for i in range(150,160):
#    print(Y_pred[i], Y_true[i])

print("Confusion Matrix:")
print(confusion_matrix(Y_true, Y_pred))

print("\n Classification Report:")
target_classes = ['No event 0 (-)', 'Event 1 (+)', 'Event 2 (+)','Event 3 (+)']
print(classification_report(Y_true, Y_pred, target_names=target_classes))


703901/703901 [==============================] - 9s 12us/step

Test Loss:  0.25346760086386555
Accuracy:   0.9999374912097013

Confusion Matrix:
[[173338      0      0     11]
 [    11 173338      0      0]
 [     0     11 173338      0]
 [     0      0     11 183843]]

 Classification Report:


ValueError: Number of classes, 4, does not match size of target_names, 3. Try specifying the labels parameter

In addition to the overal accuracy and the confusion matrix, other important criteria such as precision and recall of our classifier are shown in the table above.